# Imports & Setup

In [19]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR')

# Function definitions

In [20]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

# Main algorithm

In [22]:

print(exchange.get_positions()['NVDA'])

2023-09-16 08:00:59,054 [client    ] [MainThread  ] Unable to connect to the exchange
2023-09-16 08:00:59,055 [client    ] [MainThread  ] 
Unable to authenticate with the server. Please double-check that your username and password are correct and that you are whitelisted on an instance.
 


Exception: Unable to connect to the exchange

In [17]:
exchange = Exchange()
exchange.connect()

INSTRUMENTS = exchange.get_instruments()

QUOTED_VOLUME = 3
FIXED_MINIMUM_CREDIT = 0.07
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    print(f'')
    print(f'          (ourbid) mktbid :: mktask (ourask)')
    
    for instrument in INSTRUMENTS.values():
        # Remove all existing (still) outstanding limit orders
        exchange.delete_orders(instrument.instrument_id)
    
        # Obtain order book and only skip this instrument if there are no bids or offers available at all on that instrument,
        # as we we want to use the mid price to determine our own quoted price
        instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
        if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):
            print(f'{instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue
    
        # Obtain own current position in instrument
        position = exchange.get_positions()[instrument.instrument_id]

        # Obtain best bid and ask prices from order book to determine mid price
        best_bid_price = instrument_order_book.bids[0].price
        best_ask_price = instrument_order_book.asks[0].price
        mid_price = (best_bid_price + best_ask_price) / 2.0 
        
        # Calculate our fair/theoretical price based on the market mid price and our current position
        theoretical_price = mid_price #- PRICE_RETREAT_PER_LOT * position

        # Calculate final bid and ask prices to insert
        # bid_price = round_down_to_tick(theoretical_price - FIXED_MINIMUM_CREDIT, instrument.tick_size)
        # ask_price = round_up_to_tick(theoretical_price + FIXED_MINIMUM_CREDIT, instrument.tick_size)
        
        bid_price = theoretical_price - FIXED_MINIMUM_CREDIT
        ask_price = theoretical_price + FIXED_MINIMUM_CREDIT
        
        # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
        max_volume_to_buy = POSITION_LIMIT - position
        max_volume_to_sell = POSITION_LIMIT + position

        bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
        ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)

        # Display information for tracking the algorithm's actions
        print(f'{instrument.instrument_id:>6s} -- ({bid_price:>6.2f}) {best_bid_price:>6.2f} :: {best_ask_price:>6.2f} ({ask_price:>6.2f})')
        
        # Insert new quotes
        insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume)
        
    # Wait for a few seconds to refresh the quotes
    print(f'\nWaiting for 2 seconds.')
    time.sleep(0.05)
    
    # Clear the displayed information after waiting
    clear_output(wait=True)


-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2023-09-16 00:22:38.549476 UTC.
-----------------------------------------------------------------
Positions:
  NVDA      :  -57
  ING       :   91
  SAN       :   64
  PFE       :  -25
  CSCO      :    5

PnL: -1731.70

          (ourbid) mktbid :: mktask (ourask)
  CSCO -- ( 65.18)  65.20 ::  65.30 ( 65.32)


2023-09-16 00:22:38,844 [client    ] [Thread-15   ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


   PFE -- ( 27.53)  27.50 ::  27.70 ( 27.67)


KjException: (remote):0: failed: remote exception: expected !mWaitingToBeDisconnected; A disconnect was requested for this connection. No further messages are allowed
stack: 7fa02a3e3cc0 7fa02a3e2540 7fa02a3666d6 7fa02a354612 7fa02a354749 7fa02a349c74 7fa02a2b2639 7fa02a32b673 7fa04c91fcc0 7fa04c88fe4e 7fa03f3f4cba 7fa03f3f6293 7fa04c87f0a2 7fa04c91be68 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c925569 7fa04c925992 7fa04c91a2c6 7fa04c926b01 7fa04c87f31d 7fa04c87f421 7fa04c87f0a2 7fa04c91be68 7fa04c925569

In [2]:
!pip install ipynb

distutils: /usr/lib/python3.6/dist-packages
sysconfig: /usr/lib/python3.6/site-packages
distutils: /usr/include/python3.6/UNKNOWN
sysconfig: /usr/include/python3.6m/UNKNOWN
user = False
home = None
root = None
prefix = None
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from ipynb.fs.full.CustomBERTClassifier import predict

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [ ]:
tweet = "SocialMediaFeed15@TechBuzz: Nvidia's recent R&D project put on hold due to unexpected patent issues. A minor hiccup? #R&D #PatentIssues"
predicted_company = predict(tweet, model, tokenizer, label_encoder)
print(f"The tweet is about: {predicted_company}")